In [1]:
!wget https://github.com/rustammm/charbot-project/raw/master/dataset/filtered_big.json.zip
!tar -xf filtered_big.json.zip
!ls

--2019-04-23 17:45:40--  https://github.com/rustammm/charbot-project/raw/master/dataset/filtered_big.json.zip
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rustammm/charbot-project/master/dataset/filtered_big.json.zip [following]
--2019-04-23 17:45:40--  https://raw.githubusercontent.com/rustammm/charbot-project/master/dataset/filtered_big.json.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29622793 (28M) [application/zip]
Saving to: ‘filtered_big.json.zip.5’

filtered_big.json.z 100%[===================>]  28.25M   155MB/s    in 0.2s    

2019-04-23 17:45:41 (155 MB/s) - ‘

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask,
                            tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

    
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)


def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [0]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

    
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

    
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

    
class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

    
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)


In [0]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0


In [0]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn


In [0]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)


In [0]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))


In [0]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)


In [0]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0., max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0., d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

In [0]:
def make_model(src_vocab, tgt_vocab, N=6, 
               d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model


In [11]:
# Small example model.
tmp_model = make_model(10, 10, 2)
None

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20:

In [0]:
class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = \
                self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(
            subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask


In [0]:
def run_epoch(data_iter, model, loss_compute):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(batch.src, batch.trg, 
                            batch.src_mask, batch.trg_mask)
        if batch.ntokens == 0: continue
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 50 == 1:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                    (i, loss / batch.ntokens, tokens / elapsed))
            start = time.time()
            tokens = 0
    return total_loss / total_tokens


In [0]:
global max_src_in_batch, max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.src))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.trg) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)


In [0]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
        
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))


In [0]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.sum() > 0 and len(mask) > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))


In [0]:
def data_gen(V, batch, nbatches):
    "Generate random data for a src-tgt copy task."
    for i in range(nbatches):
        data = torch.from_numpy(np.random.randint(1, V, size=(batch, 10)))
        data[:, 0] = 1
        src = Variable(data, requires_grad=False)
        tgt = Variable(data, requires_grad=False)
        yield Batch(src, tgt, 0)

In [18]:
import sys
!{sys.executable} -m pip install git+https://github.com/Koziev/rutokenizer
!{sys.executable} -m pip install git+https://github.com/Koziev/rupostagger
!{sys.executable} -m pip install git+https://github.com/Koziev/rulemma
!{sys.executable} -m pip install python-crfsuite
!{sys.executable} -m pip install git+https://github.com/Koziev/rusyllab
!{sys.executable} -m pip install git+https://github.com/Koziev/ruword2tags

  Cloning https://github.com/Koziev/rutokenizer to /tmp/pip-req-build-j5zaic2e
  Stored in directory: /tmp/pip-ephem-wheel-cache-meh8v_pt/wheels/a6/66/ef/7c52e143b099c8aeaaf3bccc4640c065b87d7cbfb13066005d
Successfully built rutokenizer
  Cloning https://github.com/Koziev/rupostagger to /tmp/pip-req-build-2blq74wv
  Stored in directory: /tmp/pip-ephem-wheel-cache-r53sic0r/wheels/ef/af/fd/97bf178e7a272d91911645ac5ce7d7ad9c05d7cf77b8af7397
Successfully built rupostagger
  Cloning https://github.com/Koziev/rulemma to /tmp/pip-req-build-irj605qt
  Stored in directory: /tmp/pip-ephem-wheel-cache-dzjxsu4p/wheels/0b/3e/8d/0d71505e01d3f89933367910f55a366582d10c09210ecb6a4d
Successfully built rulemma
  Cloning https://github.com/Koziev/rusyllab to /tmp/pip-req-build-rbewqcxt
  Stored in directory: /tmp/pip-ephem-wheel-cache-vhrtbo4h/wheels/8b/27/a3/97cd3d0f3b259770709f14d7cb93428bbce8883028f4476b5b
Successfully built rusyllab
  Cloning https://github.com/Koziev/ruword2tags to /tmp/pip-req-build-

In [0]:
import rutokenizer
import rupostagger
import rulemma


lemmatizer = rulemma.Lemmatizer()
lemmatizer.load()

tokenizer = rutokenizer.Tokenizer()
tokenizer.load()

tagger = rupostagger.RuPosTagger()
tagger.load()

In [0]:
def preprocess_text(sent):
  tokens = tokenizer.tokenize(sent)
  tags = tagger.tag(tokens)
  lemmas = lemmatizer.lemmatize(tags)
  ret = []
  return ' '.join([lemma for word, tags, lemma, *_ in lemmas])
 


In [0]:
from tqdm import tqdm

In [22]:
preprocess_text('где')

'где'

In [23]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "<PAD>", 1: "<UNK>", 2: "<SOS>", 3 : "<EOS>", 4: "<HIS>"}
        self.n_words = 5
        self.cnt_words_total = 0

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        self.cnt_words_total += 1

        if word in self.word2index:
            self.word2count[word] += 1
        else:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
            
    def get_index_from_word(self, word):
        if word not in self.word2index:
            return self.word2index["<UNK>"]
        else:
            return self.word2index[word]
    
    def get_indexes_from_sentence(self, sentence):
        return [self.get_index_from_word(word)
            for word in sentence.split(' ')] + [3]
      
      
      
      
# For data loading.
from torchtext import data
import gc


BOS_WORD = '<s>'
EOS_WORD = '</s>'
BLANK_WORD = "<blank>"
MAX_LEN = 5
MIN_FREQ = 2




def tokenize_src(text):
    return list(filter(lambda k: len(k) > 0 ,text.split(' ')))

def tokenize_tgt(text):
    return list(filter(lambda k: len(k) > 0 ,text.split(' ')))


SRC = data.Field(tokenize=tokenize_src, pad_token=BLANK_WORD)
TGT = data.Field(tokenize=tokenize_tgt, init_token = BOS_WORD, 
                 eos_token = EOS_WORD, pad_token=BLANK_WORD)


def gen_all_data():
    global SRC, TGT
    import json
    with open('filtered_big.json', encoding='utf8') as f:
        __pairs = json.load(f)[:300000]
        
    print('read json')

    _pairs = []
    for pair in tqdm(__pairs):
        try:
            _pairs += [[preprocess_text(pair[0]), pair[1]]]
        except:
          pass
        
    print('lemmatization finished')
    l = Lang('rus')
    l2 = Lang('rus')
    for pair in _pairs:
      l.addSentence(pair[0])
      l.addSentence(pair[1])
    pairs = []
    all_ = False
    for pair in _pairs:
        add = True
        if all_:
            break
        for p in pair:
            add = add and all([l.word2count.get(x, 0) > 100 for x in p.split(' ') ])
            add = add and len(p.split(' ')) <= 5
        if add and max(len(pair[0]), len(pair[0])) > 0:
            if len(pairs) > 40000:
              all_ = True
              break
            pairs += [pair]
            l2.addSentence(pair[0])
            l2.addSentence(pair[1])
            
    print("words after:", l2.n_words)
    print("pairs after:", len(pairs))
    gc.collect()

    print(SRC)
    
    examples = [data.Example.fromlist(pair, [('src', SRC), ('trg', TGT)]) for pair in pairs]
    dataset = data.Dataset(examples, [('src', SRC), ('trg', TGT)],
                           filter_pred=lambda x: len(vars(x)['src']) <= MAX_LEN and len(vars(x)['trg']) <= MAX_LEN)
    dataset.name = ''
    train, val, test = dataset.split(split_ratio=[0.8,0.1,0.1])
    
    SRC.build_vocab(train.src, min_freq=MIN_FREQ)
    TGT.build_vocab(train.trg, min_freq=MIN_FREQ)
    return train, val, test
  
train, val, test = gen_all_data()
gc.collect()

  0%|          | 76/300000 [00:00<06:36, 756.83it/s]

read json


  2%|▏         | 5792/300000 [00:07<06:08, 798.41it/s]

неизвестный тэг "Case=Voc"


  2%|▏         | 6473/300000 [00:08<05:49, 840.38it/s]

...

100%|█████████▉| 299710/300000 [06:34<00:00, 733.58it/s]

неизвестный тэг "Case=Voc"
неизвестный тэг "Case=Voc"
неизвестный тэг "Case=Voc"
неизвестный тэг "Case=Voc"
неизвестный тэг "Case=Voc"


100%|██████████| 300000/300000 [06:35<00:00, 758.98it/s]


lemmatization finished
words after: 1628
pairs after: 40001


0

In [0]:
import json
with open("SRCVS.json", 'w') as f:
    json.dump(SRC.vocab.stoi, f)

In [0]:
with open("TGTVI.json", 'w') as f:
    json.dump(TGT.vocab.itos, f)

In [0]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out = model.decode(memory, src_mask, 
                           Variable(ys), 
                           Variable(subsequent_mask(ys.size(1))
                                    .type_as(src.data)))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.data[0]
        ys = torch.cat([ys, 
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys



In [0]:
class MyIterator(data.Iterator):
    def create_batches(self):
        if self.train:
            def pool(d, random_shuffler):
                for p in data.batch(d, self.batch_size * 100):
                    p_batch = data.batch(
                        sorted(p, key=self.sort_key),
                        self.batch_size, self.batch_size_fn)
                    for b in random_shuffler(list(p_batch)):
                        yield b
            self.batches = pool(self.data(), self.random_shuffler)
            
        else:
            self.batches = []
            for b in data.batch(self.data(), self.batch_size,
                                          self.batch_size_fn):
                self.batches.append(sorted(b, key=self.sort_key))

def rebatch(pad_idx, batch):
    "Fix order in torchtext to match ours"
    src, trg = batch.src.transpose(0, 1), batch.trg.transpose(0, 1)
    return Batch(src, trg, pad_idx)


In [26]:
# GPUs to use
devices = [0]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if True:
    pad_idx = TGT.vocab.stoi["<blank>"]
    model = make_model(len(SRC.vocab), len(TGT.vocab), N=6)
    model.cuda()
    criterion = LabelSmoothing(size=len(TGT.vocab), padding_idx=pad_idx, smoothing=0.1)
    criterion.cuda()
    #model_par = nn.DataParallel(model, device_ids=devices)
None


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20:

In [0]:
BATCH_SIZE = 8000
train_iter = MyIterator(train, batch_size=BATCH_SIZE, device=device,
                        repeat=False, sort_key=lambda x: (len(x.src), len(x.trg)),
                        batch_size_fn=batch_size_fn, train=True)
valid_iter = MyIterator(val, batch_size=BATCH_SIZE, device=device,
                        repeat=False, sort_key=lambda x: (len(x.src), len(x.trg)),
                        batch_size_fn=batch_size_fn, train=False)

In [31]:
!nvidia-smi

Tue Apr 23 17:55:32 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    29W /  70W |    941MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), 
                              y.contiguous().view(-1)) / norm
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return loss.data * norm


In [33]:
if True:
    model_opt = NoamOpt(model.src_embed[0].d_model, 1, 2000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
    for epoch in range(10):
        model.train()
        run_epoch((rebatch(pad_idx, b) for b in train_iter), 
                  model, 
                  SimpleLossCompute(model.generator, criterion, model_opt))
        model.eval()
        loss = run_epoch((rebatch(pad_idx, b) for b in valid_iter), 
                          model, 
                          SimpleLossCompute(model.generator, criterion, None))
        print(loss)


Epoch Step: 1 Loss: 6.543178 Tokens per Sec: 10240.000000
Epoch Step: 1 Loss: 4.918178 Tokens per Sec: 7165.000000
tensor(5.0114, device='cuda:0')
Epoch Step: 1 Loss: 5.062703 Tokens per Sec: 9435.000000
Epoch Step: 1 Loss: 4.049404 Tokens per Sec: 7165.000000
tensor(4.1369, device='cuda:0')
Epoch Step: 1 Loss: 3.659666 Tokens per Sec: 7934.000000
Epoch Step: 1 Loss: 3.693854 Tokens per Sec: 7165.000000
tensor(3.7768, device='cuda:0')
Epoch Step: 1 Loss: 3.364217 Tokens per Sec: 8457.000000
Epoch Step: 1 Loss: 3.519328 Tokens per Sec: 7165.000000
tensor(3.6249, device='cuda:0')
Epoch Step: 1 Loss: 3.281430 Tokens per Sec: 9729.000000
Epoch Step: 1 Loss: 3.366447 Tokens per Sec: 7165.000000
tensor(3.4533, device='cuda:0')
Epoch Step: 1 Loss: 3.812054 Tokens per Sec: 10631.000000
Epoch Step: 1 Loss: 3.257100 Tokens per Sec: 7165.000000
tensor(3.3789, device='cuda:0')
Epoch Step: 1 Loss: 3.752457 Tokens per Sec: 10789.000000
Epoch Step: 1 Loss: 3.163717 Tokens per Sec: 7165.000000
tensor(

In [56]:
for epoch in range(40):
        model.train()
        run_epoch((rebatch(pad_idx, b) for b in train_iter), 
                  model, 
                  SimpleLossCompute(model.generator, criterion, model_opt))
        model.eval()
        loss = run_epoch((rebatch(pad_idx, b) for b in valid_iter), 
                          model, 
                          SimpleLossCompute(model.generator, criterion, None))
        print(loss)

Epoch Step: 1 Loss: 1.789357 Tokens per Sec: 9112.000000
Epoch Step: 1 Loss: 1.210122 Tokens per Sec: 7165.000000
tensor(1.1366, device='cuda:0')
Epoch Step: 1 Loss: 1.773655 Tokens per Sec: 9389.000000
Epoch Step: 1 Loss: 1.144369 Tokens per Sec: 7165.000000
tensor(1.0804, device='cuda:0')
Epoch Step: 1 Loss: 1.907395 Tokens per Sec: 11703.000000
Epoch Step: 1 Loss: 1.156643 Tokens per Sec: 7165.000000
tensor(1.0985, device='cuda:0')
Epoch Step: 1 Loss: 2.021931 Tokens per Sec: 11544.000000
Epoch Step: 1 Loss: 1.108849 Tokens per Sec: 7165.000000
tensor(1.0622, device='cuda:0')
Epoch Step: 1 Loss: 1.761966 Tokens per Sec: 12026.000000
Epoch Step: 1 Loss: 1.096713 Tokens per Sec: 7165.000000
tensor(1.0361, device='cuda:0')
Epoch Step: 1 Loss: 1.708932 Tokens per Sec: 10644.000000
Epoch Step: 1 Loss: 1.016617 Tokens per Sec: 7165.000000
tensor(0.9863, device='cuda:0')
Epoch Step: 1 Loss: 1.524658 Tokens per Sec: 10802.000000
Epoch Step: 1 Loss: 0.990902 Tokens per Sec: 7165.000000
tenso

In [57]:
for i, batch in enumerate(valid_iter):
    src = batch.src.transpose(0, 1)[:1]
    
    print('Query:', end="\t")
    for i in range(1, batch.src.size(0)):
        sym = SRC.vocab.itos[batch.src.data[i, 0]]
        if sym == "</s>": break
        print(sym, end =" ")
    print()
            
    src_mask = (src != SRC.vocab.stoi["<blank>"]).unsqueeze(-2)
    out = greedy_decode(model, src, src_mask, 
                        max_len=60, start_symbol=TGT.vocab.stoi["<s>"])
    
    print("Translation:", end="\t")
    for i in range(1, out.size(1)):
        sym = TGT.vocab.itos[out[0, i]]
        if sym == "</s>": break
        print(sym, end =" ")
    print()
    print("Target:", end="\t")
    for i in range(1, batch.trg.size(0)):
        sym = TGT.vocab.itos[batch.trg.data[i, 0]]
        if sym == "</s>": break
        print(sym, end =" ")
    print()
    if i > 20:
        break

Query:	ключ 
Translation:	я замок 
Target:	я замок 
Query:	дело ? 
Translation:	нормально 
Target:	все в порядке 
Query:	этот женщина ? 
Translation:	она - моя госпожа 
Target:	она - моя госпожа 
Query:	нужный ли помощь ? 
Translation:	но оно все мало ! 
Target:	но оно все мало ! 


In [0]:
def average(model, models):
    "Average models into model"
    for ps in zip(*[m.params() for m in [model] + models]):
        p[0].copy_(torch.sum(*ps[1:]) / len(ps[1:]))


In [61]:
torch.save(model, 'chitchat_best.pt')
#torch.save(model_par, 'chitchat2.pt')
!ls

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type EncoderDecoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Encoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type EncoderLayer. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type MultiHeadedAttention. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/us

adc.json	   filtered_big.json.zip    filtered_big.json.zip.4
chitchat3.pt	   filtered_big.json.zip.1  filtered_big.json.zip.5
chitchat_best.pt   filtered_big.json.zip.2  sample_data
filtered_big.json  filtered_big.json.zip.3


In [0]:
!ls -lh

In [0]:
#!pip install -U -q PyDrive

import tensorflow as tf
import timeit

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)




In [0]:
uploaded = drive.CreateFile({'chitchat_best.pt': 'chitchat_best.pt'})
uploaded.SetContentFile('chitchat_best.pt')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))


In [73]:
!ls

adc.json	   filtered_big.json.zip    filtered_big.json.zip.4  TGTVI.json
chitchat3.pt	   filtered_big.json.zip.1  filtered_big.json.zip.5  TRGVI.json
chitchat_best.pt   filtered_big.json.zip.2  sample_data
filtered_big.json  filtered_big.json.zip.3  SRCVS.json


In [76]:
uploaded = drive.CreateFile({'SRCVS.json': 'SRCVS.json'})
uploaded.SetContentFile('SRCVS.json')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1iS9mTJZmITnL6RX68HP7lhqAAhuq-0cG


In [77]:
uploaded = drive.CreateFile({'TGTVI.json': 'TGTVI.json'})
uploaded.SetContentFile('TGTVI.json')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1BwYjcjdbEtVCn77N6kI-JUOMR1DV0qNO


In [79]:
TGT.vocab.stoi["<s>"]

2

In [0]:
def eval(text):
  model.eval()
  sent = text.split()
  src = torch.LongTensor([[SRC.vocab.stoi[w] for w in sent]])
  src = Variable(src).cuda()
  src_mask = (src != SRC.vocab.stoi["<blank>"]).unsqueeze(-2)
  out = greedy_decode(model, src, src_mask, 
                      max_len=60, start_symbol=TGT.vocab.stoi["<s>"])
  #print("Translation:", end="\t")
  trans = "<s> "
  for i in range(1, out.size(1)):
      sym = TGT.vocab.itos[out[0, i]]
      if sym == "</s>": break
      trans += sym + " "
  return trans


In [66]:
tests2 = ['Привет',
      'Как тебя зовут?',
      'Как дела?',
      'У меня тоже нормально?',
      'Чего такой молчаливый?',
      'Понятно',
      'Пошли гулять?',
      'Куда?',
      'Говорят в Москве много где погулять можно',
      'Да здесь!',
      'Что думаешь?']



for t in [preprocess_text('Иду за тобой'), preprocess_text('иду гулять')]:
    print(eval(t))
    
for t in tests2:
    print(t, eval(preprocess_text(t)))


<s> кто ты ? 
<s> ага 
Привет <s> привет 
Как тебя зовут? <s> я не знаю 
Как дела? <s> нормально 
У меня тоже нормально? <s> да 
Чего такой молчаливый? <s> это ты мне скажи 
Понятно <s> да нет босс 
Пошли гулять? <s> пошли 
Куда? <s> домои 
Говорят в Москве много где погулять можно <s> где она ? 
Да здесь! <s> что здесь ? 
Что думаешь? <s> ничего 


In [0]:
tgt_sent = trans.split()
def draw(data, x, y, ax):
    seaborn.heatmap(data, 
                    xticklabels=x, square=True, yticklabels=y, vmin=0.0, vmax=1.0, 
                    cbar=False, ax=ax)
    
for layer in range(1, 6, 2):
    fig, axs = plt.subplots(1,4, figsize=(20, 10))
    print("Encoder Layer", layer+1)
    for h in range(4):
        draw(model.encoder.layers[layer].self_attn.attn[0, h].data, 
            sent, sent if h ==0 else [], ax=axs[h])
    plt.show()
    
for layer in range(1, 6, 2):
    fig, axs = plt.subplots(1,4, figsize=(20, 10))
    print("Decoder Self Layer", layer+1)
    for h in range(4):
        draw(model.decoder.layers[layer].self_attn.attn[0, h].data[:len(tgt_sent), :len(tgt_sent)], 
            tgt_sent, tgt_sent if h ==0 else [], ax=axs[h])
    plt.show()
    print("Decoder Src Layer", layer+1)
    fig, axs = plt.subplots(1,4, figsize=(20, 10))
    for h in range(4):
        draw(model.decoder.layers[layer].self_attn.attn[0, h].data[:len(tgt_sent), :len(sent)], 
            sent, tgt_sent if h ==0 else [], ax=axs[h])
    plt.show()


In [0]:
!ls -lh